<a href="https://colab.research.google.com/github/urpolina/Big-data---Spark/blob/main/Urvantseva_DSU_35_BD_%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Первая часть. Python

###1. Загрузка файлов по оценкам (ratings) и фильмам (movies)

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('u.data.csv', sep='\t', header=0, names=['user id', 'item id', 'rating', 'timestamp'])
ratings.head()

,user id,item id,rating,timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [3]:
movies_col = ['movie id', 'movie title', 'release date', 'video release date',
              'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children',
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [5]:
movies = pd.read_csv('u.item.csv', sep='|', header=0, names=movies_col, encoding = 'ISO-8859-1')
movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###2. Пользователь, поставивший больше всего оценок

In [6]:
filtr = ratings['user id'].value_counts().index[0]
filtr

405

###3. Датафрейм ratings только по тем фильмы, который оценил данный пользователь

In [8]:
ratings_filtr = ratings[['user id', 'item id', 'rating']][ratings['user id'] == filtr]
ratings_filtr

,user id,item id,rating
12275,405,56,4
12382,405,592,1
12429,405,1582,1
12448,405,171,1
12459,405,580,1
...,...,...,...
98955,405,375,1
98977,405,445,4
99147,405,1246,1
99464,405,196,1


###4. Добавим к датафрейму из п. 3 следующие столбцы:
* По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
* столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей

In [9]:
merg = ratings_filtr.merge(movies[['movie id', 'release date', 'unknown', 'Action', 'Adventure', 'Animation', 'Children',
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
              'Sci-Fi', 'Thriller', 'War', 'Western']], left_on='item id', right_on='movie id', how='left')
merg.head()

,user id,item id,rating,movie id,release date,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,56,01-Jan-1994,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,592,1,592,01-Jan-1995,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,1582,01-Jan-1947,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,405,171,1,171,01-Jan-1991,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,405,580,1,580,01-Jan-1995,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
merg['release date'] = pd.to_datetime(merg['release date'])

In [11]:
merg['year'] = merg['release date'].dt.year
merg.head()

,user id,item id,rating,movie id,release date,unknown,Action,Adventure,Animation,Children,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,405,56,4,56,1994-01-01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
1,405,592,1,592,1995-01-01,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1995
2,405,1582,1,1582,1947-01-01,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1947
3,405,171,1,171,1991-01-01,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1991
4,405,580,1,580,1995-01-01,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1995


In [12]:
ratings_sum = ratings[['item id', 'rating']].groupby(['item id']).agg(['count', 'sum'])
ratings_sum.head()

rating      
         count   sum
item id             
1          452  1753
2          131   420
3           90   273
4          209   742
5           86   284

In [13]:
merg_sum = merg.merge(ratings_sum, on='item id', how='left')
merg_sum.head()

<ipython-input-13-8790937678fb>:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  merg_sum = merg.merge(ratings_sum, on='item id', how='left')


,user id,item id,rating,movie id,release date,unknown,Action,Adventure,Animation,Children,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,"(rating, count)","(rating, sum)"
0,405,56,4,56,1994-01-01,0,0,0,0,0,...,0,0,0,0,0,0,0,1994,394,1600
1,405,592,1,592,1995-01-01,0,0,0,0,0,...,0,1,0,0,1,0,0,1995,9,30
2,405,1582,1,1582,1947-01-01,0,0,0,0,0,...,0,0,0,0,0,0,0,1947,1,1
3,405,171,1,171,1991-01-01,0,0,0,0,0,...,0,0,0,1,0,0,0,1991,65,252
4,405,580,1,580,1995-01-01,0,0,0,0,0,...,0,0,1,0,0,0,0,1995,32,108


In [14]:
df = merg_sum.drop(['movie id','release date'], axis=1)
df.head()

,user id,item id,rating,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,"(rating, count)","(rating, sum)"
0,405,56,4,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1994,394,1600
1,405,592,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1995,9,30
2,405,1582,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1947,1,1
3,405,171,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1991,65,252
4,405,580,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1995,32,108


In [15]:
df = df.rename(columns={('rating', 'count'): 'count', ('rating', 'sum'): 'sum'})

In [16]:
df.columns

Index(['user id', 'item id', 'rating', 'unknown', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'year', 'count', 'sum'],
      dtype='object')

In [17]:
df.columns = df.columns.astype(str)

###5. Сформируем X_train, X_test, y_train, y_test

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X, y = df[['unknown','Action','Adventure','Animation','Children','Comedy',
           'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror',
           'Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western',
           'year','count','sum']], df['rating']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

###6. Обучим модель линейной регрессии на фильмах

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
model = LinearRegression()

In [23]:
model.fit(X_train, y_train)

LinearRegression()

In [24]:
y_pred = model.predict(X_test)
y_pred

array([1.80007526, 1.18727266, 2.62865332, 1.64343417, 1.22794724,
       1.19067904, 1.15738422, 2.81556953, 1.50671106, 1.26453624,
       0.9443175 , 1.11896653, 1.22097977, 1.26878056, 2.47310432,
       2.87672195, 1.48728854, 0.80457523, 3.76953055, 2.7241943 ,
       1.28078407, 1.37345888, 1.77887536, 1.10726418, 1.09280039,
       2.11518569, 2.99773663, 1.37777239, 1.21108119, 1.27802389,
       4.06762047, 2.70404642, 2.0730247 , 2.15436302, 3.11124377,
       1.91364616, 1.14364142, 1.49505922, 1.94880816, 0.80115878,
       2.86281285, 1.3850365 , 1.1687017 , 3.46840283, 2.51719796,
       1.60314058, 1.16134558, 2.30425191, 3.38586718, 1.10704292,
       1.33897347, 1.16714854, 2.67314034, 1.19959611, 1.29259368,
       2.41340223, 1.17816697, 1.28804642, 1.64393646, 1.97048545,
       3.39560921, 3.39543984, 1.18915927, 1.18896576, 1.25365754,
       3.20711247, 2.65704201, 1.22041429, 2.30021481, 3.60374358,
       1.53303117, 2.22936988, 1.14658697, 3.82353567, 5.20735

###7. Оценим качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_test, y_pred) # чем лучше предсказание, тем ближе значение к 0

1.237362774602054

##Вторая часть. Spark

###8. Загрузим данные в spark

In [27]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,242 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,016 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main 

In [39]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [40]:
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [41]:
!tar -xvf spark-3.5.0-bin-hadoop3.tgz

spark-3.5.0-bin-hadoop3/
spark-3.5.0-bin-hadoop3/kubernetes/
spark-3.5.0-bin-hadoop3/kubernetes/tests/
spark-3.5.0-bin-hadoop3/kubernetes/tests/pyfiles.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/autoscale.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/python_executable_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/worker_memory_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/py_container_checks.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning_cleanup.py
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/decom.sh
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/python/
spark-3.5.0-bin-hadoop3/kubern

In [42]:
!pip install -q findspark

In [46]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [47]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [48]:
df_ratings = spark.read.csv('u.data.csv', sep='\t', header=False, inferSchema=True)
df_ratings = df_ratings.toDF('user id', 'item id', 'rating', 'timestamp')
df_ratings.show(5)

+-------+-------+------+---------+
|user id|item id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [49]:
df_movies = spark.read.csv('u.item.csv', sep='|', header=False, inferSchema=True)
df_movies = df_movies.toDF('movie id', 'movie title', 'release date', 'video release date',
              'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children',
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western')
df_movies.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie id|      movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              NULL|http

###9. Средствами спарка выведем среднюю оценку для каждого фильма

In [50]:
df_ratings.groupBy('item id').avg('rating').show()

+-------+------------------+
|item id|       avg(rating)|
+-------+------------------+
|    496| 4.121212121212121|
|    471|3.6108597285067874|
|    463| 3.859154929577465|
|    148|          3.203125|
|   1342|               2.5|
|    833| 3.204081632653061|
|   1088| 2.230769230769231|
|   1591|3.1666666666666665|
|   1238|             3.125|
|   1580|               1.0|
|   1645|               4.0|
|    392|3.5441176470588234|
|    623| 2.923076923076923|
|    540| 2.511627906976744|
|    858|               1.0|
|    737| 2.983050847457627|
|    243|2.4393939393939394|
|   1025|2.9318181818181817|
|   1084| 3.857142857142857|
|   1127| 2.909090909090909|
+-------+------------------+
only showing top 20 rows



###10. Средствами спарка посчитаем среднюю оценку для каждого жанра

In [51]:
from pyspark.sql.functions import avg, col

In [52]:
genres = ['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [53]:
for genre in genres:
    genre_df = df_movies.where(col(genre) == 1)
    genre_ratings_df = genre_df.join(df_ratings, genre_df['movie id'] == df_ratings['item id'])
    avg_rating = genre_ratings_df.select(avg('rating')).collect()[0][0]
    print(genre, avg_rating)

unknown 3.2
Action 3.480245417953027
Adventure 3.503526503308369
Animation 3.5766990291262135
Children 3.3532442216652742
Comedy 3.3940734781442745
Crime 3.6322780881440098
Documentary 3.6728232189973613
Drama 3.6873793708484772
Fantasy 3.2152366863905324
Film-Noir 3.9215233698788228
Horror 3.2903893172841827
Musical 3.521396851029471
Mystery 3.63813155386082
Romance 3.621704948358255
Sci-Fi 3.5607227022780834
Thriller 3.5090069495245064
War 3.815811874866993
Western 3.6132686084142396


###11. В спарке получим 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами по количеству оценок

In [54]:
from pyspark.sql.functions import desc

In [55]:
# 5 самых популярных фильмов по количеству оценок
df_ratings.groupBy('item id').count().sort(desc('count')).show(5)

+-------+-----+
|item id|count|
+-------+-----+
|     50|  583|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
+-------+-----+
only showing top 5 rows



In [56]:
# 5 самых непопулярных фильмов по количеству оценок
df_ratings.groupBy('item id').count().sort('count').show(5)

+-------+-----+
|item id|count|
+-------+-----+
|   1460|    1|
|   1618|    1|
|   1339|    1|
|   1645|    1|
|   1650|    1|
+-------+-----+
only showing top 5 rows

